Visualizing the network models to generate clusters
- Get a pretrained vgg with some of its training data from ECCV that has code/data (>10classes).
- Insert probes at each layer to capture all neuron outputs (activations)
- Set a pruning threshold without losing accuracy 
    - Eg 10 exmaples for one class, keep pruning until prediction barely changes
- Run images from each class through the network, grab outputs above a certain threshold as key value (key being the network layer concat neuron number and value is the activation
- Check how many in each category is shared by images from the same category
- Check how many are shared in all category/some of the categories
- Plot histogram of # of neurons shared by n categories (also per layer)
- Look at percentage pruning for each class (hopefully >90%)
- We do knowledge distillation from each class starting from some base layer (to be varied)

In [1]:
import sys
import torch
import torch.nn as nn
import numpy as np
from PIL import Image as im

YOLOV3_HOME = '/homes/iws/emazuh/projects/animal_detection/yolov3'
sys.path.append(YOLOV3_HOME)
LABELING_TOOL_HOME = '/homes/iws/emazuh/projects/animal_detection/labelling_tool'
sys.path.append(LABELING_TOOL_HOME)
sys.path.append('/homes/iws/emazuh/miniconda3/lib/python3.8/site-packages/torch')

from yolov3_models import Darknet
from yolov3_utils.datasets import LoadImagesAndLabels
from yolov3_utils.utils import non_max_suppression, torch_utils, load_classes
from yolov3_utils.parse_config import parse_data_cfg
from tracking.Utilities import drawBox, cv2

In [2]:
img_size = 416
batch_size = 1
WEIGHTS_PATH = '/local1/emazuh/elk/yolo/yolov3/weights/acamp4k8_tiny'
net_cfg = YOLOV3_HOME + '/cfg/elk-tiny-yolov3.cfg'# '/cfg/elk_yolov3.cfg'
weights = WEIGHTS_PATH + '/best_99.pt' #'/best_359.pt'
test_path = '/local1/emazuh/elk/yolo/yolov3/data/animals/output/test_inv.txt'

device = torch_utils.select_device()
model = Darknet(net_cfg, img_size).to(device)
model.load_state_dict(torch.load(weights, map_location=device)['model'])

# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)

model.eval()
dataset = LoadImagesAndLabels(test_path, img_size, batch_size, rect=False, sort_files=True)

Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', total_memory=11178MB)
           device1 _CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', total_memory=11178MB)

Reading annotations from /local1/emazuh/elk/yolo/yolov3/data/animals/output/test_inv.txt
Sorting image files...
Found 99 sequences with 120 annotations


In [ ]:
Image("/local1/emazuh/elk/yolo/yolov3/data/animals/elk/elk_10_w/image000241.jpg")

In [3]:
animals = ["bear", "moose", "coyote", "deer", "elk", "bison"]
exs = {animal: [] for animal in animals}
for i in range(120):
    label = dataset[i][3]
    a = label.split('/')[-3]
    exs[a].append(dataset[i][1:])

In [4]:
device = torch_utils.select_device()

Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', total_memory=11178MB)
           device1 _CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', total_memory=11178MB)



In [96]:
outputs = []
animal = 'elk'
for i in range(2):
    _, _, probe = model(exs[animal][i][0].unsqueeze(0).to(device))
    outputs.append(probe)

module Sequential(
  (yolo_16): YOLOLayer()
)
module Sequential(
  (yolo_23): YOLOLayer()
)
module Sequential(
  (yolo_16): YOLOLayer()
)
module Sequential(
  (yolo_23): YOLOLayer()
)


In [72]:
# o, p, probe = model(exs['elk'][0][0].unsqueeze(0).to(device))

In [85]:
# bear_outputs = outputs

In [90]:
b_act_1 = bear_outputs[0][10]['activation']
b_act_2 = bear_outputs[1][10]['activation']
b_flat_act_1 = b_act_1.reshape(-1)
b_flat_act_2 = b_act_2.reshape(-1)

In [91]:
b_flat_act_1.dot(b_flat_act_2)

tensor(24.78176, device='cuda:0', grad_fn=<DotBackward>)

In [92]:
((b_flat_act_1 > 1e-32)*(b_flat_act_2 > 1e-32)).cpu().detach().numpy().mean()

0.17112379807692307

In [102]:
((b_flat_act_1 > 1e-32)*(flat_act_2 > 1e-32)).cpu().detach().numpy().mean()

0.20088295118343194

In [97]:
# elk_outputs = outputs
# first only compare activations on first layer
act_1 = elk_outputs[0][10]['activation']
act_2 = elk_outputs[1][10]['activation']

In [98]:
flat_act_1 = act_1.reshape(-1)
flat_act_2 = act_2.reshape(-1)

In [104]:
flat_act_1.dot(flat_act_2)

tensor(29.25773, device='cuda:0', grad_fn=<DotBackward>)

In [99]:
flat_act_1.dot(flat_act_2)

tensor(29.25773, device='cuda:0', grad_fn=<DotBackward>)

In [100]:
((flat_act_1 > 1e-32)*(flat_act_2 > 1e-32)).cpu().detach().numpy().mean()

0.19371764053254437

In [24]:
(probe[0]['params'][0][1].cpu().detach().numpy() > 0.0000001).mean()

0.49074074074074076

In [28]:
probe[0]['activation'].cpu().detach().numpy().shape

(1, 16, 416, 416)

In [29]:
(probe[0]['activation'].cpu().detach().numpy() > 0.0000001).mean()

0.4147154678254438

In [45]:
act = probe[0]['activation']#.cpu().detach().numpy()

In [74]:
len(probe)

24

In [37]:
# for each example
#  for each layer
#.  flatten activation (neuron_name = layer_flatid)
act.resize(act.size)

In [48]:
flat_act = act.reshape((-1))

In [51]:
act_d = flat_act.dot(flat_act)

In [55]:
act_d

tensor(6234343.50000, device='cuda:0', grad_fn=<DotBackward>)

In [56]:
# consider gpu kmeans 
# https://github.com/subhadarship/kmeans_pytorch

In [70]:
(flat_act < 1e-32).sum()/flat_act.size()[0]

tensor(0.58528, device='cuda:0')

In [ ]:
# for two examples, see where overlap of zeros is